# Smartcab from OpenAI Gym

Code taken from (https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/), using as a guide to practice and understand.

"There are 4 locations (labeled by different letters), and our job is to pick up the passenger at one location and drop him off at another. We receive +20 points for a successful drop-off and lose 1 point for every time-step it takes. There is also a 10 point penalty for illegal pick-up and drop-off actions."

In [1]:
import gym

In [5]:
env = gym.make("Taxi-v3").env
env.reset()
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [11]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)
env.s = state
env.render()

State: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [12]:
# Each reward value structure at a certain state: {action: [(probability, nextstate, reward, done)]}
# States as rows and actions as columns.
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

## Without Reinforcement Learning

In [13]:
env.s = 328
epochs = 0
penalties, reward = 0, 0

frames = []

done = False

while not done:
    # Selects one random action from all possible actions
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 857
Penalties incurred: 296


In [21]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 857
State: 0
Action: 5
Reward: 20


### Summary
The agent takes a lot of timesteps before dropping the passenger off at the right location. 
This is happening because the agent is not storing past actions to inform decisions moving forward. 

## With Reinforcement Learning

Q-learning introduces the Q-value, which uses the environment's rewards to learn over time. The values are stored in a Q-table which maps to each (state, action) pair and updates based on the reward taken from P for each action.

Utilizing the Q-table, the agent will likely know the reward for performing "pick-up" when the passenger is at their current location is higher than other actions like "drop off"

In [31]:
import numpy as np
# Q-tables are initialized as zero then updates after every action.
q_table = np.zeros([env.observation_space.n, env.action_space.n])
print(q_table)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


### Training the agent

In [33]:
%%time

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1 # Learning rate: extent to whch new information overrides old information, 0 = exploit -> 1 = explore.
gamma = 0.6 # Discount factor: how much importance we give future rewards, 0 = immediate -> 1 = future.
epsilon = 0.1 # Greediness factor

# For plotting metrics
all_epochs = []
all_penalties = []

# 100,000 episodes
for i in range(1, 100001):
    state = env.reset()
    
    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    # Determines which action to take by random choice
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values
        
        # Sets the next state as the observation of the environment 
        next_state, reward, done, info = env.step(action) 
        
        # Storing the Q-value of the current state and action taken
        old_value = q_table[state, action]
        # Takes the max Q-value of all actions available based on the current state
        next_max = np.max(q_table[next_state])
        
        # Computes the new value of the current state
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        # Adjusts the Q-value
        q_table[state, action] = new_value
        
        if reward == -10:
            penalties += 1
        
        # Sets the current state as the next state
        state = next_state
        epochs += 1

    if i % 100 == 0:
        clear_output(wait=True)
        # Q-table updating after every episode.
        print(q_table)
        print(f"Episode: {i}")
        
print("Training finished.\n")

[[  0.           0.           0.           0.           0.
    0.        ]
 [ -2.41837066  -2.3639511   -2.41837066  -2.3639511   -2.27325184
  -11.3639511 ]
 [ -1.870144    -1.45024     -1.870144    -1.45024     -0.7504
  -10.45024   ]
 ...
 [ -0.99674129   0.416       -0.99429209  -1.27803266  -6.04341703
   -8.32753283]
 [ -2.15957885  -2.1220864   -2.18909716  -2.1220864   -8.26841755
   -5.13979007]
 [  4.46221724   1.81752735   5.24855987  11.          -2.95546735
   -3.0092534 ]]
Episode: 100000
Training finished.

CPU times: user 27.7 s, sys: 2.65 s, total: 30.4 s
Wall time: 28.4 s


In [24]:
q_table[328]

array([ -2.40166752,  -2.27325184,  -2.404524  ,  -2.36045761,
       -10.32359512, -10.75220972])

The max Q-value from the initial state in the array is -2.27, which means 'north' is the most optimal route.

## Evaluating the agent
Using the Q-table from training.

In [34]:
trained_frames = []

total_epochs, total_penalties = 0, 0
# Episodes are how many "rounds" we want the agent to go through until the goal is complete.
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        # The first action will be a random one from the Q-table.
        action = np.argmax(q_table[state])
        
        state, reward, done, info = env.step(action)
        
        if reward == -10:
            penalties += 1

        # Put each rendered frame into dict for animation
        trained_frames.append({
            'frame': env.render(mode='ansi'),
            'state': state,
            'action': action,
            'reward': reward
            }
        )
        
        epochs += 1
    
    total_epochs += epochs
    total_penalties += penalties

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 13.61
Average penalties per episode: 0.0


### Visualization of the SmartCab using Q-learning

In [35]:
from IPython.display import clear_output
from time import sleep
print_frames(trained_frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 1361
State: 85
Action: 5
Reward: 20
